first we import the modules that we going to use :

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import sklearn as sk 
from pandas.plotting import scatter_matrix
from statsmodels.graphics.tsaplots import plot_acf

for each swimmer S, we  have a periode a velocity a flagellum_length and a classe Fresh_only (A), Frozen_only (C) or Blebbed (B)

we merged all data in a file csv and then we use a Dataframe all_data by extracting the file

In [ ]:
all_data = pd.read_csv("all_data.csv")

we calculate the velocity of the swimmer by extracting the min and max amplitude of the mid-point, we then calculte distance between them and finally divide the result by the period of the swimmer 

In [3]:
def Velocity(data):
    Y = data["Unnamed: 1500"]
    P = (np.abs(max(Y)-min(Y))/2)/data["period"].dropna()
    return P.iloc[0]

In [5]:
V = []
for i in range(1,217):
    path = "all_data/"+str(i)+".csv"
    df = pd.read_csv(path)
    Velo = Velocity(df)
    V.append(Velo)



We extracte the class of each swimmer

In [ ]:
classe = pd.read_csv("class.csv")

def func(df,i):
    if df.iloc[i]["fresh"] == 1:
        return "A"
    elif df.iloc[i]["frozen"] == 1 and df.iloc[i]["blebbed"]:
        return "B"
    else: 
        return "C"
    
classe_np = [func(classe,item) for item in range(0,216)]
classe_np


We create a new data with the four variable T.P.V.C

In [ ]:

New_all_data = pd.Dataframe({"periode" : list(all_data["period"].dropna()),
        "flagellum_length" : list(all_data["flagellum_length"].dropna()),
        "Velocity" : V, 
        "Class" : classe_np})

Here we use K-mean cluster to classifie data 

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans 
from sklearn.datasets import load_digits

spespe_A = Class_A[["Velocity","periode","flagellum_length"]]
a = np.array([spespe_A.iloc[0]])
for i in range(1,79):
    a = np.vstack([a,spespe_A.iloc[i]])

pca = PCA(2)
df = pca.fit_transform(a)


kmeans = KMeans(n_clusters= 2)
#predict the labels of clusters.
label = kmeans.fit_predict(df)


u_labels = np.unique(label)

